Импортируем все, что нам нужно:

In [39]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import time

Загружаем датасет:

In [40]:
data = pd.read_csv('diabetes.csv')    # считываем данные из csv файла
data                                  # проверяем, что данные считались нормально

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


Разбиваем данные на тестовую и тренировочную выборки:

In [41]:
x = data.drop(['Outcome'], axis=1)                                                                         # выбираем признаки
y = data['Outcome']                                                                                        # выбираем целевую переменную
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=0)       # разделяем данные на тестовую и тренировочную выборки

Возьмем модель из лабораторной 5.2:

In [42]:
bst = XGBClassifier(learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8, random_state=121)   #
start_time = time.time()                                                                                 #
bst.fit(x_train, y_train)                                                                                #
time_for_fit = time.time() - start_time                                                                  #  моедль из лабораторной 5.2
preds = bst.predict(x_test)                                                                              #
print('F1: ', f1_score(y_test, preds))                                                                   #
print('Время обучения: ', time_for_fit)                                                                  #

F1:  0.6918238993710691
Время обучения:  0.05884194374084473


Теперь подберем гиперпараметры XGBoost с помощью Skikit-Learn Random Search:

In [59]:
from sklearn.model_selection import RandomizedSearchCV

bst = XGBClassifier(random_state=121)  # создаем модель
distributions = dict(max_depth=range(1, 21), num_estimators=range(10, 210, 10),
					 subsample=np.arange(0, 1, 0.1), reg_lambda=range(0, 100, 10), reg_alpha=range(0, 100, 10))       # список возможных значений параметров модели
clf = RandomizedSearchCV(bst, distributions, random_state=0, scoring="f1")                 # ищем лучшие параметры случайным поиском
start_time = time.time()                                                                   # время начала обучения
search = clf.fit(x_train, y_train)                                                         # обучаем модель
search_time = time.time() - start_time                                                     # ищем затраченное время
print('Подобранные гиперпараметры: ', search.best_params_)                                 # выврдим набор подобранных параметров
print('Полученный F1: ', search.best_score_)                                               # выводим результат F1
print('Затраченное время: ', search_time)                                                  # выводим затраченное время на обучение

Подобранные гиперпараметры:  {'subsample': np.float64(0.30000000000000004), 'reg_lambda': 80, 'reg_alpha': 0, 'num_estimators': 190, 'max_depth': 18}
Полученный F1:  0.6336016525863424
Затраченное время:  1.7110188007354736


Теперь подберем гиперпараметры, используя TPE и Hyperopt:

In [ ]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

def objective(params):                                                    # принцип, по которому выбираются лучшие гиперпараметры
	model = XGBClassifier(**params)
	model.fit(x_train, y_train)
	pred = model.predict(x_test)
	return {'loss': -f1_score(y_test, pred), 'status': STATUS_OK}

space = {'max_depth': hp.choice('max_depth', range(1, 21)),
		 'n_estimators': hp.choice('n_estimators', range(10, 210, 10)),
		 'subsample': hp.uniform('subsample', 0, 1),
		 'reg_lambda': hp.choice('reg_lambda', range(0, 100, 10)),
		 'reg_alpha': hp.choice('reg_alpha', range(0, 100, 10))}          # список возможных значений парамеров

In [66]:
trials = Trials()                                                                    # все результаты обучения
start_time = time.time()                                                             # время на начало обучения
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=40,
			trials=trials, show_progressbar=False)                                   # поиск лучших гиперпараметров
search_time = time.time() - start_time                                               # время конца обучения
best_params = {'max_depth': list(range(1, 21))[best['max_depth']],
			   'n_estimators': list(range(10, 210, 10))[best['n_estimators']],
			   'subsample': best['subsample'],
			   'reg_lambda': list(range(0, 100, 10))[best['reg_lambda']],
			   'reg_aplha': list(range(0, 100, 10))[best['reg_alpha']]}              # представляем параметры в удобной форме
print('Подобранные гиперпараметры: ', best_params)                                   # выводим подобранный набор гиперпараметров
print('Полученные F1: ', -trials.best_trial['result']['loss'])                       # выводим значение F1 для данного набора
print('Затраченное время: ', search_time)                                            # выводим затраченное время

Подобранные гиперпараметры:  {'max_depth': 7, 'n_estimators': 120, 'subsample': np.float64(0.8848605065236543), 'reg_lambda': 10, 'reg_aplha': 0}
Полученные F1:  0.7125
Затраченное время:  2.0265798568725586


Таким образом, за время чуть больше чем при случайном поиске мы получили F1 качество лучше, чем при случайном поиске. При случайном поиске получает, что reg_alpha стремится быть очень маленьким (около 0), reg_lambda стремится к 80, количество деревьев большое (т.е. 190), глубина деревьев равна 18. При поиске с помощью TPE, получаем меньшую глубину (7 по сравнению с 18) и меньшее число деревьев (120 по сравнению с 190), а также меньшее значение reg_lambda = 10. Reg_alpha осталось таким же.